<a href="https://colab.research.google.com/github/jetsonmom/6.23_automobility_lesson/blob/main/OpenCV%EB%A1%9C_Canny_%2B_Hough%EB%A1%9C_%EC%B0%A8%EC%84%A0_%EA%B0%90%EC%A7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step 1: OpenCV로 Canny + Hough로 차선 감지

Step 2: ROI 적용 및 영상 처리

Step 3: CNN을 사용한 간단한 edge 분류 (후속 실습)

In [ ]:
# 구글 드라이브 마운트 (선택)
from google.colab import drive
drive.mount('/content/drive')

# OpenCV 설치
!pip install opencv-python
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
# yt-dlp 설치
!pip install yt-dlp

# 원하는 유튜브 영상 다운로드
!yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 https://www.youtube.com/shorts/s5ynrR1x9ig


! 업로드하신 영상(mp4 등) 파일은 cv2.imread()로는 읽을 수 없고, **cv2.VideoCapture()**를 사용해서 프레임 단위로 추출해야

In [ ]:
import cv2
from google.colab import files
import matplotlib.pyplot as plt

# 영상 파일 업로드
uploaded = files.upload()

# 파일 이름 가져오기
video_path = list(uploaded.keys())[0]
print("업로드한 영상:", video_path)

# OpenCV로 영상 열기
cap = cv2.VideoCapture(video_path)

# 첫 프레임 읽기
ret, frame = cap.read()
cap.release()

if not ret:
    print("❌ 영상을 읽을 수 없습니다.")
else:
    # 프레임을 출력해보기
    plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    plt.title("첫 번째 프레임")
    plt.axis('off')
    plt.show()


In [ ]:
첫 프레임만 차선 인식

In [ ]:
import cv2
from google.colab import files
import matplotlib.pyplot as plt
import numpy as np

# 영상 업로드
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 영상에서 첫 프레임 가져오기
cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()
cap.release()

if not ret:
    raise ValueError("영상을 읽을 수 없습니다.")

# 1. 그레이스케일 변환
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 2. 블러 → 에지(Canny)
blur = cv2.GaussianBlur(gray, (5, 5), 0)
edges = cv2.Canny(blur, 50, 150)

# 3. ROI 설정 (영상 하단 절반)
height, width = edges.shape
mask = np.zeros_like(edges)
polygon = np.array([[
    (0, height),
    (width, height),
    (width, int(height * 0.6)),
    (0, int(height * 0.6))
]])
cv2.fillPoly(mask, polygon, 255)
roi = cv2.bitwise_and(edges, mask)

# 4. Hough Transform으로 직선 검출
lines = cv2.HoughLinesP(roi, 2, np.pi / 180, threshold=50, minLineLength=40, maxLineGap=50)

# 5. 원본 프레임에 선 그리기
line_image = frame.copy()
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

# 6. 결과 출력
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("원본 프레임")

plt.subplot(1, 2, 2)
plt.imshow(cv2.cvtColor(line_image, cv2.COLOR_BGR2RGB))
plt.title("차선 인식 결과")
plt.axis('off')
plt.show()


In [ ]:
import cv2
import numpy as np
from google.colab import files

# 영상 업로드
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 원본 영상 읽기
cap = cv2.VideoCapture(video_path)

# 영상 정보 가져오기
width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps    = cap.get(cv2.CAP_PROP_FPS)

# 결과 저장용 비디오 라이터 설정
out = cv2.VideoWriter('lane_detected_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 1. 그레이 변환 & 블러 & 에지
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blur, 50, 150)

    # 2. ROI 마스크
    mask = np.zeros_like(edges)
    polygon = np.array([[
        (0, height),
        (width, height),
        (width, int(height * 0.6)),
        (0, int(height * 0.6))
    ]])
    cv2.fillPoly(mask, polygon, 255)
    roi = cv2.bitwise_and(edges, mask)

    # 3. 허프 변환
    lines = cv2.HoughLinesP(roi, 2, np.pi / 180, threshold=50, minLineLength=40, maxLineGap=50)

    # 4. 차선 그리기
    line_image = frame.copy()
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line[0]
            cv2.line(line_image, (x1, y1), (x2, y2), (0, 255, 0), 5)

    # 5. 프레임 저장
    out.write(line_image)

cap.release()
out.release()
print("✅ 완료: lane_detected_output.mp4 저장됨!")
